# Env

In [20]:
import argparse
import collections
import os
import random
import re
import shutil
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import torch

In [2]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# kowiki file 경로저장
kowiki_file = "/content/drive/MyDrive/Data/nlp_corpus/kowiki.txt.zip"

In [4]:
# 환경 설정
args = {
    # random seed value
    "seed": 1234,
    # CPU 또는 GPU 사용여부 결정
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    # 말뭉치 파일
    "corpus": kowiki_file,
}
args = argparse.Namespace(**args)

print(args)

Namespace(corpus='/content/drive/MyDrive/Data/nlp_corpus/kowiki.txt.zip', device=device(type='cpu'), seed=1234)


In [5]:
# random seed 설정
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

# 파일확인

In [6]:
# wiki 라인수 확인
count = 0
with zipfile.ZipFile(args.corpus) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            count += 1
print(count)

3724301


In [7]:
# wiki 내용 확인
with zipfile.ZipFile(args.corpus) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            if i >= 50:
                break
            line = line.decode('utf-8').strip()
            print(line)

지미 카터
제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.
1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워, 포드를 누르고 당선되었다.
카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.
카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다.
그러나 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 1979년 백악관에서 양국 간의 평화조약으로 이끌어졌다. 또한 소련과 제2차 전략 무기 제한 협상에 조인했다.
카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.
그러나 주 이란 미국 대사관 인질 사건에서 인질 구출 실패를 이유로 1980년 대통령 선거에서 공화당의 로널드 레이건 후보에게 져 결국 재선에 실패했다. 또한 임기 말기에 터진 소련의 아프가니스탄 침공 사건으로 인해 1980년 하계 올림픽에 반공국가들의 보이콧을 내세웠다.
지미 카터는 대한민국과의 관계에서도 중요한 영향을 미쳤던 대통

# Char Tokenizer

In [ ]:
char_counter = collections.defaultdict(int)
# char 개수 확인
with zipfile.ZipFile(args.corpus) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            line = line.decode('utf-8').strip()
            for c in line:
                char_counter[c] += 1

In [ ]:
# char 개수
print(len(char_counter))

15581


In [ ]:
# 빈도수가 많은 글자 10개 출력
most_freq = sorted(char_counter.items(), key=lambda item: item[1], reverse=True)

print(most_freq[:10])

[(' ', 55863103), ('이', 5799143), ('다', 5291578), ('.', 4395303), ('에', 4313327), ('의', 4228709), ('는', 3848984), ('로', 3112533), (',', 3110187), ('하', 2989928)]


In [ ]:
# 빈도수가 적은 글자 10개 출력
least_freq = sorted(char_counter.items(), key=lambda item: item[1])

print(least_freq[:10])

[('蔟', 1), ('蓓', 1), ('踢', 1), ('ǣ', 1), ('钊', 1), ('臄', 1), ('饀', 1), ('蝌', 1), ('廏', 1), ('Ꝩ', 1)]


In [ ]:
# token 개수
count = 0
for c, cnt in char_counter.items():
    count += cnt
print(count)

267502382


In [ ]:
# char에 일련번호 부여
char_to_id = {'[PAD]': 0, '[UNK]': 1}
for c, cnt in char_counter.items():
    char_to_id[c] = len(char_to_id)
print(len(char_to_id))

15583


In [ ]:
# wiki char tokenize
with zipfile.ZipFile(args.corpus) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            if i >= 5:
                break
            line = line.decode('utf-8').strip()
            print(line)
            _id = [char_to_id[c] for c in line]
            print(_id)

지미 카터
[2, 3, 4, 5, 6]
제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.
[7, 8, 9, 4, 10, 4, 11, 2, 3, 11, 4, 5, 6, 4, 12, 13, 14, 15, 16, 4, 17, 18, 19, 20, 21, 4, 17, 22, 23, 4, 17, 24, 4, 25, 4, 26, 27, 4, 28, 12, 29, 4, 30, 31, 4, 3, 32, 4, 33, 18, 34, 35, 4, 36, 37, 38, 4, 15, 17, 18, 39, 39, 21, 4, 25, 4, 17, 18, 40, 17, 21, 26, 41, 42, 43]
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
[2, 3, 4, 5, 6, 27, 4, 44, 2, 45, 12, 4, 46, 6, 4, 5, 47, 48, 4, 49, 50, 51, 9, 4, 52, 53, 54, 55, 4, 56, 14, 57, 42, 43, 4, 44, 2, 45, 4, 58, 59, 36, 60, 61, 62, 4, 63, 64, 65, 66, 42, 43, 4, 67, 4, 68, 4, 69, 70, 54, 4, 71, 14, 72, 4, 73, 74, 75, 76, 77, 78, 75, 79, 80, 74, 81, 4, 82, 83, 76, 84, 85, 4, 24, 65, 66, 42, 43, 4, 17, 18, 86, 33, 21, 4, 3, 32, 4, 69, 70, 4, 36, 87, 85, 4, 88, 89, 65, 66, 90, 4, 41, 68, 4, 91, 92, 75, 93, 94, 4, 95, 5

# Word Tokenizer

In [ ]:
word_counter = collections.defaultdict(int)
# word 개수 확인
with zipfile.ZipFile(args.corpus) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            line = line.decode('utf-8').strip()
            for w in line.split():
                word_counter[w] += 1

In [ ]:
# word 개수
print(len(word_counter))

6714211


In [ ]:
# 빈도수가 많은 단어 10개 출력
most_freq = sorted(word_counter.items(), key=lambda item: item[1], reverse=True)

print(most_freq[:10])

[('있다.', 465195), ('이', 272738), ('수', 238266), ('있는', 195963), ('그', 193328), ('그는', 111749), ('한다.', 109327), ('~', 107753), ('되었다.', 103153), ('다른', 101086)]


In [ ]:
# 빈도수가 적은 단어 10개 출력
least_freq = sorted(word_counter.items(), key=lambda item: item[1])

print(least_freq[:10])

[('1981년)이다.', 1), ('전함·원자력·잠수함의', 1), ('땅콩·면화', 1), ('농부"', 1), ('(Peanut', 1), ('Farmer)로', 1), ('낙선하나', 1), ('연임했으며,', 1), ('반공국가들의', 1), ('정보기관·의회의', 1)]


In [ ]:
# token 개수
count = 0
for c, cnt in word_counter.items():
    count += cnt
print(count)

58205952


In [ ]:
# word에 일련번호 부여
word_to_id = {'[PAD]': 0, '[UNK]': 1}
for w, cnt in word_counter.items():
    word_to_id[w] = len(word_to_id)
print(len(word_to_id))

6714213


In [ ]:
# wiki word tokenize
with zipfile.ZipFile(args.corpus) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            if i >= 5:
                break
            line = line.decode('utf-8').strip()
            print(line)
            _id = [word_to_id[w] for w in line.split()]
            print(_id)

지미 카터
[2, 3]
제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.
[4, 5, 6, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 11, 19]
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
[2, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 15, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]
1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.
[55, 27, 56, 57, 58, 59, 60, 30, 61, 62, 63, 64, 65, 66, 67, 27, 56, 68, 69, 70, 71, 27, 56, 72, 73, 74, 75, 76, 21, 77, 78, 79, 80, 81, 27, 82, 83, 27, 84, 85, 86, 87, 88, 89, 90]
1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워, 포드를 누르고 당선되었다.
[91, 17, 69, 13, 92, 93, 94, 95, 96, 97, 98, 99]


# Morp Tokenizer

In [52]:
# 형태소분석기 설치
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
     |████████████████████████████████| 19.4MB 44.8MB/s 
     |████████████████████████████████| 460kB 42.3MB/s 
     |████████████████████████████████| 92kB 6.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ bash -x
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|mint' /etc/lsb-release /etc/os-release
+ '[' '/etc/lsb-release:DISTRIB_ID=Ubuntu
/etc/lsb-release:DISTRIB_DESCRIPTION="Ubuntu 18.04.5

In [53]:
import konlpy
# mecab 선언
mecab = konlpy.tag.Mecab()

In [ ]:
morph_counter = collections.defaultdict(int)
# morph 개수 확인
with zipfile.ZipFile(args.corpus) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            line = line.decode('utf-8').strip()
            for m in mecab.morphs(line):
                morph_counter[m] += 1

In [ ]:
# morph 개수
print(len(morph_counter))

660409


In [ ]:
# 빈도수가 많은 morph 10개 출력
most_freq = sorted(morph_counter.items(), key=lambda item: item[1], reverse=True)

print(most_freq[:10])

[('.', 4380736), ('의', 3641831), ('다', 3459076), ('이', 3332539), ('는', 3307338), (',', 3053205), ('에', 2755034), ('을', 2577961), ('하', 2348573), ('은', 1916337)]


In [ ]:
# 빈도수가 적은 morph 10개 출력
least_freq = sorted(morph_counter.items(), key=lambda item: item[1])

print(least_freq[:10])

[('mathmaticus', 1), ('mathematikos', 1), ('mathema', 1), ('manthanein', 1), ('춤춘다는', 1), ('isobar', 1), ('太蔟', 1), ('變徵', 1), ('三遷齋', 1), ('一小學校', 1)]


In [ ]:
# token 개수
count = 0
for c, cnt in morph_counter.items():
    count += cnt
print(count)

129449561


In [ ]:
# morph에 일련번호 부여
morph_to_id = {'[PAD]': 0, '[UNK]': 1}
for w, cnt in morph_counter.items():
    morph_to_id[w] = len(morph_to_id)
print(len(morph_to_id))

660411


In [ ]:
# wiki morph tokenize
with zipfile.ZipFile(args.corpus) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            if i >= 5:
                break
            line = line.decode('utf-8').strip()
            print(line)
            morphs = mecab.morphs(line)
            print(morphs)
            _id = [morph_to_id[m] for m in mecab.morphs(line)]
            print(_id)

지미 카터
['지미', '카터']
[2, 3]
제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.
['제임스', '얼', '"', '지미', '"', '카터', '주니어', '(', ',', '1924', '년', '10', '월', '1', '일', '~', ')', '는', '민주당', '출신', '미국', '39', '번', '째', '대통령', '(', '1977', '년', '~', '1981', '년', ')', '이', '다', '.']
[4, 5, 6, 2, 6, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 8, 26, 11, 16, 27, 11, 17, 28, 29, 30]
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
['지미', '카터', '는', '조지', '아주', '섬터', '카운티', '플', '레인스', '마을', '에서', '태어났', '다', '.', '조지', '아', '공과', '대학교', '를', '졸업', '하', '였', '다', '.', '그', '후', '해군', '에', '들어가', '전함', '·', '원자력', '·', '잠수함', '의', '승무원', '으로', '일', '하', '였', '다', '.', '1953', '년', '미국', '해군', '대위', '로', '예편', '하', '였', '고', '이후', '땅콩', '·', '면화', '등', '을', '가꿔', '많', '은', '돈', '을', '벌', '었', '다', '.', '그', '의', '별명', '

# BPE

In [ ]:
# BPE corpus
corpus = """
low lower newest widest
low lower newest widest
low       newest widest
low       newest
low       newest
          newest
"""

In [ ]:
word_counter = collections.defaultdict(int)
# word 개수 확인
with zipfile.ZipFile(args.corpus) as z:
    for w in corpus.strip().split():
        word_counter[w] += 1

print(word_counter)

defaultdict(<class 'int'>, {'low': 5, 'lower': 2, 'newest': 6, 'widest': 3})


In [ ]:
bpe_counter = collections.defaultdict(int)

# subword 개수 확인
for w, n in word_counter.items():
    w = f"\u2581{w}"
    bpe_counter[" ".join(w)] = n

print(bpe_counter)

defaultdict(<class 'int'>, {'▁ l o w': 5, '▁ l o w e r': 2, '▁ n e w e s t': 6, '▁ w i d e s t': 3})


In [ ]:
def update_vocab(vocab, counter):
    """
    vocab 변경
    :param vocab: vocabulary
    :param counter: BPE counter
    """
    for w in counter:
        for s in w.split():
            if s not in vocab:
                vocab[s] = len(vocab)
    return vocab

In [ ]:
# bpe 일련번호 부여
bpe_to_id = {'[PAD]': 0, '[UNK]': 1}
bpe_to_id = update_vocab(bpe_to_id, bpe_counter)

print(bpe_to_id)

{'[PAD]': 0, '[UNK]': 1, '▁': 2, 'l': 3, 'o': 4, 'w': 5, 'e': 6, 'r': 7, 'n': 8, 's': 9, 't': 10, 'i': 11, 'd': 12}


In [ ]:
def get_stats(counter):
    """
    bi-gram 빈도수 계산
    :param counter: BPE counter
    """
    pairs = collections.defaultdict(int)
    for word, freq in counter.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

In [ ]:
def merge_vocab(pair, v_in):
    """
    bi-gram merge
    :param counter: BPE counter
    """
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

In [ ]:
# bigram pair
pairs = get_stats(bpe_counter)

print(pairs)

defaultdict(<class 'int'>, {('▁', 'l'): 7, ('l', 'o'): 7, ('o', 'w'): 7, ('w', 'e'): 8, ('e', 'r'): 2, ('▁', 'n'): 6, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('▁', 'w'): 3, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3})


In [ ]:
# find most freq bigram pair
best = max(pairs, key=pairs.get)

print(best)

('e', 's')


In [ ]:
# merge most freq bigram pair
bpe_counter = merge_vocab(best, bpe_counter)

print(bpe_counter)

{'▁ l o w': 5, '▁ l o w e r': 2, '▁ n e w es t': 6, '▁ w i d es t': 3}


In [ ]:
# update vocab
bpe_to_id = update_vocab(bpe_to_id, bpe_counter)

print(bpe_to_id)

{'[PAD]': 0, '[UNK]': 1, '▁': 2, 'l': 3, 'o': 4, 'w': 5, 'e': 6, 'r': 7, 'n': 8, 's': 9, 't': 10, 'i': 11, 'd': 12, 'es': 13}


In [ ]:
# bigram pair
pairs = get_stats(bpe_counter)
print(pairs)
# find most freq bigram pair
best = max(pairs, key=pairs.get)
print(best)
# merge most freq bigram pair
bpe_counter = merge_vocab(best, bpe_counter)
print(bpe_counter)
# update vocab
bpe_to_id = update_vocab(bpe_to_id, bpe_counter)
print(bpe_to_id)

defaultdict(<class 'int'>, {('▁low', 'e'): 2, ('e', 'r'): 2, ('▁', 'n'): 6, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'est'): 6, ('▁', 'w'): 3, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est'): 3})
('▁', 'n')
{'▁low': 5, '▁low e r': 2, '▁n e w est': 6, '▁ w i d est': 3}
{'[PAD]': 0, '[UNK]': 1, '▁': 2, 'l': 3, 'o': 4, 'w': 5, 'e': 6, 'r': 7, 'n': 8, 's': 9, 't': 10, 'i': 11, 'd': 12, 'es': 13, 'est': 14, '▁l': 15, '▁lo': 16, '▁low': 17, '▁n': 18}


# Sentencepiece

In [16]:
# Sentencepiece 설치
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 4.4MB/s 


In [17]:
# import sentencepiece
import sentencepiece as spm

In [18]:
def train_sentencepiece(corpus, prefix, vocab_size=32000):
    """
    sentencepiece를 이용해 vocab 학습
    :param corpus: 학습할 말뭉치
    :param prefix: 저장할 vocab 이름
    :param vocab_size: vocab 개수
    """
    spm.SentencePieceTrainer.train(
        f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" +  # 7은 특수문자 개수
        " --model_type=unigram" +
        " --max_sentence_length=999999" +  # 문장 최대 길이
        " --pad_id=0 --pad_piece=[PAD]" +  # pad token 및 id 지정
        " --unk_id=1 --unk_piece=[UNK]" +  # unknown token 및 id 지정
        " --bos_id=2 --bos_piece=[BOS]" +  # begin of sequence token 및 id 지정
        " --eos_id=3 --eos_piece=[EOS]" +  # end of sequence token 및 id 지정
        " --user_defined_symbols=[SEP],[CLS],[MASK]" +  # 기타 추가 토큰 SEP: 4, CLS: 5, MASK: 6
        " --input_sentence_size=100000" +  # 말뭉치에서 셈플링해서 학습
        " --shuffle_input_sentence=true")  # 셈플링한 말뭉치 shuffle

In [15]:
# 현재 작업폴더에 압축풀기
!unzip /content/drive/MyDrive/Data/nlp_corpus/kowiki.txt.zip

print(os.listdir("./"))

Archive:  /content/drive/MyDrive/Data/nlp_corpus/kowiki.txt.zip
  inflating: kowiki.txt              
  inflating: __MACOSX/._kowiki.txt   
['.config', '.ipynb_checkpoints', 'drive', 'kowiki.txt', '__MACOSX', 'sample_data']


In [19]:
# vocab 학습
train_sentencepiece("kowiki.txt", "kowiki_32000")

In [21]:
# 생성결과 확인
print(os.listdir("./"))

['.config', '.ipynb_checkpoints', 'drive', 'kowiki.txt', 'kowiki_32000.model', 'kowiki_32000.vocab', '__MACOSX', 'sample_data']


In [27]:
# 생성파일 백업
corpus_dir = os.path.dirname(args.corpus)
shutil.copy("kowiki_32000.model", corpus_dir)
shutil.copy("kowiki_32000.vocab", corpus_dir)

print(os.listdir(corpus_dir))

['kowiki.txt.zip', 'kowiki_32000.model', 'kowiki_32000.vocab']


In [28]:
# load vocab
spm_vocab = spm.SentencePieceProcessor()
spm_vocab.load(os.path.join(corpus_dir, "kowiki_32000.model"))

True

In [30]:
# wiki spm tokenize
with zipfile.ZipFile(args.corpus) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            if i >= 5:
                break
            line = line.decode('utf-8').strip()
            print(line)
            tokens = spm_vocab.encode_as_pieces(line)
            print(tokens)
            _ids = spm_vocab.encode_as_ids(line)
            print(_ids)

지미 카터
['▁지미', '▁카터']
[14443, 19046]
제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.
['▁제임스', '▁얼', '▁"', '지', '미', '"', '▁카터', '▁주니어', '(', ',', '▁1924', '년', '▁10', '월', '▁1', '일', '▁~', '▁)', '는', '▁민주당', '▁출신', '▁미국', '▁39', '번째', '▁대통령', '▁', '(1977', '년', '▁~', '▁1981', '년', ')', '이다', '.']
[2000, 5505, 49, 44, 170, 50, 19046, 5788, 16, 9, 5550, 17, 67, 24, 36, 30, 86, 140, 12, 2475, 704, 139, 5014, 672, 654, 10, 26475, 17, 86, 2282, 17, 13, 26, 7]
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
['▁지미', '▁카터', '는', '▁조지아주', '▁섬', '터', '▁카운티', '▁플레', '인', '스', '▁마을에서', '▁태어났다', '.', '▁조지아', '▁공과대학교', '를', '▁졸업', '하였다', '.', '▁그', '▁후', '▁해군', '에', '▁들어가', '▁전함', '·', '원자력', '·', '잠', '수', '함', '의', '▁승무원', '으로', '▁일', '하였다', '.', '▁1953', '년', '▁미국', '▁해군', '▁대위', '로', '▁예편', '하였고', '▁이후', '▁땅콩', '·', '면', '화', '▁등을', '▁가', '꿔', 

In [44]:
# 문자열을 token으로 분할
tokens = spm_vocab.encode_as_pieces("아름다운 대한민국 우리나라 금수강산")

print(tokens)

['▁아름다운', '▁대한민국', '▁우리나라', '▁금', '수', '강', '산']


In [49]:
# token을 문자열로 복원
print(spm_vocab.decode_pieces(tokens))

아름다운 대한민국 우리나라 금수강산


In [46]:
# 문자열을 숫자로 분할
ids = spm_vocab.encode_as_ids("아름다운 대한민국 우리나라 금수강산")

print(ids)

[4869, 243, 6308, 653, 104, 301, 162]


In [48]:
# 숫자를 문자열로 복원
print(spm_vocab.decode_ids(ids))

아름다운 대한민국 우리나라 금수강산


In [50]:
# token을 숫자로 변경
print(spm_vocab.piece_to_id(tokens))

[4869, 243, 6308, 653, 104, 301, 162]


In [51]:
# 숫자를 token으로 변경
print(spm_vocab.id_to_piece(ids))

['▁아름다운', '▁대한민국', '▁우리나라', '▁금', '수', '강', '산']


# Sentencepiece with Morph

In [56]:
# morph 단위로 분할된 말뭉치 생성
with open("kowiki-mecab.txt", "w") as o_f:
    with zipfile.ZipFile(args.corpus) as z:
        with z.open('kowiki.txt') as f:
            for i, line in enumerate(f):
                line = line.decode('utf-8').strip()
                tokens = mecab.morphs(line)
                string = " ".join(tokens)
                o_f.write(string)
                o_f.write("\n")

In [57]:
# 파일 확인
print(os.listdir("./"))

['.config', 'kowiki-mecab.txt', '.ipynb_checkpoints', 'drive', 'kowiki.txt', 'kowiki_32000.model', 'kowiki_32000.vocab', '__MACOSX', 'sample_data']


In [58]:
# morph vocab 학습
train_sentencepiece("kowiki-mecab.txt", "kowiki_mecab_32000")

In [59]:
# 생성결과 확인
print(os.listdir("./"))

['.config', 'kowiki_mecab_32000.vocab', 'kowiki_mecab_32000.model', 'kowiki-mecab.txt', '.ipynb_checkpoints', 'drive', 'kowiki.txt', 'kowiki_32000.model', 'kowiki_32000.vocab', '__MACOSX', 'sample_data']


In [60]:
# 생성파일 백업
corpus_dir = os.path.dirname(args.corpus)
shutil.copy("kowiki_mecab_32000.model", corpus_dir)
shutil.copy("kowiki_mecab_32000.vocab", corpus_dir)

print(os.listdir(corpus_dir))

['kowiki.txt.zip', 'kowiki_32000.model', 'kowiki_32000.vocab', 'kowiki_mecab_32000.model', 'kowiki_mecab_32000.vocab']


In [61]:
# load morph vocab
spm_morph_vocab = spm.SentencePieceProcessor()
spm_morph_vocab.load(os.path.join(corpus_dir, "kowiki_mecab_32000.model"))

True

In [62]:
# wiki spm morph tokenize
with zipfile.ZipFile(args.corpus) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            if i >= 5:
                break
            line = line.decode('utf-8').strip()
            print(line)
            string = " ".join(mecab.morphs(line))
            print(string)
            tokens = spm_morph_vocab.encode_as_pieces(string)
            print(tokens)
            _ids = spm_morph_vocab.encode_as_ids(string)
            print(_ids)

지미 카터
지미 카터
['▁지', '미', '▁카', '터']
[45, 534, 556, 57]
제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.
제임스 얼 " 지미 " 카터 주니어 ( , 1924 년 10 월 1 일 ~ ) 는 민주당 출신 미국 39 번 째 대통령 ( 1977 년 ~ 1981 년 ) 이 다 .
['▁제임스', '▁얼', '▁"', '▁지', '미', '▁"', '▁카', '터', '▁주니어', '▁(', '▁,', '▁192', '4', '▁년', '▁10', '▁월', '▁1', '▁일', '▁~', '▁)', '▁는', '▁민주', '당', '▁출신', '▁미국', '▁39', '▁번', '▁째', '▁대통령', '▁(', '▁1977', '▁년', '▁~', '▁198', '1', '▁년', '▁)', '▁이', '▁다', '▁.']
[2279, 3285, 51, 45, 534, 51, 556, 57, 4215, 19, 12, 668, 160, 21, 125, 35, 50, 36, 114, 18, 11, 904, 229, 656, 139, 2540, 128, 268, 513, 19, 2435, 21, 114, 194, 389, 21, 18, 10, 9, 7]
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
지미 카터 는 조지 아주 섬터 카운티 플 레인스 마을 에서 태어났 다 . 조지 아 공과 대학교 를 졸업 하 였 다 . 그 후 해군 에 들어가 전함 · 원자력 · 잠수함 의 승무원 으로 일 하 였 다 . 1953 년 미국 해군 대위 로 예편 하 였 고 이후 땅콩 · 면화 등 을 가꿔 많 은 돈